In [144]:
%matplotlib inline
import pandas
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from torch.utils.data import Dataset
from IPython.display import display
from collections import defaultdict
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

In [111]:
data_df = pd.read_csv('clean-phone-data-for-students.csv')

In [112]:
display(data_df.head())
data_df.describe()

,Sentence Utterance,Action,Object
0,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,enquire,payment
1,internet ยังความเร็วอยุ่เท่าไหร ครับ,enquire,package
2,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้...,report,suspend
3,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อ...,enquire,internet
4,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโ...,report,phone_issues


,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


In [113]:
data_df = data_df[["Sentence Utterance", "Object"]]
data_df.columns = ['input', 'raw_label']
display(data_df.describe())
display(data_df.raw_label.unique())

,input,raw_label
count,16175,16175
unique,13389,33
top,บริการอื่นๆ,service
freq,97,2525


array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nonTrueMove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd',
       'TrueMoney', 'garbage', 'Payment', 'IDD', 'ringtone', 'Idd',
       'rate', 'loyalty_card', 'contact', 'officer', 'Balance', 'Service',
       'Loyalty_card'], dtype=object)

In [114]:
data_df['clean_label']=data_df['raw_label'].str.lower().copy()
data_df.drop('raw_label', axis=1, inplace=True)
display(data_df.describe())
display(data_df.clean_label.unique())

,input,clean_label
count,16175,16175
unique,13389,26
top,บริการอื่นๆ,service
freq,97,2528


array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nontruemove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd', 'garbage',
       'ringtone', 'rate', 'loyalty_card', 'contact', 'officer'],
      dtype=object)

In [115]:
data_df = data_df.drop_duplicates("input", keep="first")
data_df = data_df.reset_index(drop=True)
display(data_df.describe())

,input,clean_label
count,13389,13389
unique,13389,26
top,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,service
freq,1,2111


# Model 1 TF-IDF

Build a model to train a tf-idf text classifier. Use a simple logistic regression model for the classifier.

For this part, you may find this [tutorial](https://scikit-learn.org/stable/auto_examples/text/plot_document_classification_20newsgroups.html#sphx-glr-auto-examples-text-plot-document-classification-20newsgroups-py) helpful.

Below are some design choices you need to consider to accomplish this task. Be sure to answer them when you submit your model.

What tokenizer will you use? Why?

**Ans:**

Will you ignore some stop words (a, an, the, to, etc. for English) in your tf-idf? Is it important?
PythaiNLP provides a list of stopwords if you want to use (https://pythainlp.org/docs/2.0/api/corpus.html#pythainlp.corpus.common.thai_stopwords)

**Ans:**

The dictionary of TF-IDF is usually based on the training data. How many words in the test set are OOVs?

**Ans:**

In [116]:
import deepcut
import attacut
import time
from tqdm import tqdm
import pythainlp

In [117]:
# 19 secs for 100 samples
# 13389 samples = 42 mins; ain't nobody got time for that
start_time = time.time()
for i in range(100):
    # print(f"original: {data_df['input'][i]}")
    # print(deepcut.tokenize(data_df['input'][i]))
    deepcut.tokenize(data_df['input'][i])
end_time = time.time()

print(f"Total time: {end_time - start_time}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
3/3 ━━━━━━━

In [118]:
# 2.25 secs for 100 samples
# 13389 samples = 5 mins; now we're talking
start_time = time.time()
for i in range(100):
    attacut.tokenize(data_df['input'][0])
end_time = time.time()

print(f"Total time: {end_time - start_time}")


/Users/idhibhatpankam/Code/courses/NLP-SYS/.venv/lib/python3.12/site-packages/attacut/models/__init__.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_d

Total time: 3.103543996810913


In [119]:
data = data_df.copy().to_numpy()

unique_label = data_df.clean_label.unique()

label_2_num_map = dict(zip(unique_label, range(len(unique_label))))
num_2_label_map = dict(zip(range(len(unique_label)), unique_label))

print("Create Mappings")
display(num_2_label_map)
display(label_2_num_map)

# print("Before Mappings")
# display(data[:, 1])
data[:,1] = np.vectorize(label_2_num_map.get)(data[:,1])
# print("After Mappings")
# display(data[:, 1])

Create Mappings


{0: 'payment',
 1: 'package',
 2: 'suspend',
 3: 'internet',
 4: 'phone_issues',
 5: 'service',
 6: 'nontruemove',
 7: 'balance',
 8: 'detail',
 9: 'bill',
 10: 'credit',
 11: 'promotion',
 12: 'mobile_setting',
 13: 'iservice',
 14: 'roaming',
 15: 'truemoney',
 16: 'information',
 17: 'lost_stolen',
 18: 'balance_minutes',
 19: 'idd',
 20: 'garbage',
 21: 'ringtone',
 22: 'rate',
 23: 'loyalty_card',
 24: 'contact',
 25: 'officer'}

{'payment': 0,
 'package': 1,
 'suspend': 2,
 'internet': 3,
 'phone_issues': 4,
 'service': 5,
 'nontruemove': 6,
 'balance': 7,
 'detail': 8,
 'bill': 9,
 'credit': 10,
 'promotion': 11,
 'mobile_setting': 12,
 'iservice': 13,
 'roaming': 14,
 'truemoney': 15,
 'information': 16,
 'lost_stolen': 17,
 'balance_minutes': 18,
 'idd': 19,
 'garbage': 20,
 'ringtone': 21,
 'rate': 22,
 'loyalty_card': 23,
 'contact': 24,
 'officer': 25}

In [120]:
def strip_str(string):
    return string.strip()
     
# print("Before")
# print(data)
data[:,0] = np.vectorize(strip_str)(data[:,0])
# print("After")
# print(data)

In [121]:
data

array([['<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counter Services เค้าเช็ต 3276.25 บาท เมื่อวานที่ผมเช็คที่ศูนย์บอกมียอด 3057.79 บาท',
        0],
       ['internet ยังความเร็วอยุ่เท่าไหร ครับ', 1],
       ['ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้ ค่ะ', 2],
       ...,
       ['ยอดเงินเหลือเท่าไหร่ค่ะ', 7],
       ['ยอดเงินในระบบ', 7],
       ['สอบถามโปรโมชั่นปัจจุบันที่ใช้อยู่ค่ะ', 1]], dtype=object)

In [122]:
x = data[:, 0]
y = data[:, 1]
print(x.shape, y.shape)

(13389,) (13389,)


In [160]:
train_X: np.ndarray
test_X: np.ndarray
train_y: np.ndarray
test_y: np.ndarray

train_X, test_X, train_y, test_y = train_test_split(x, y, stratify=y, test_size=0.2,random_state=42)
print(train_X.shape, test_X.shape, train_y.shape, test_y.shape)

(10711,) (2678,) (10711,) (2678,)


In [161]:
vocab = defaultdict(lambda: len(vocab))
vocab["<UNK>"] = 0
for i in tqdm(range(len(train_X))):
    tokens = attacut.tokenize(train_X[i])
    for token in tokens:
        vocab[token]
    
# 4728 unique tokens in training set (+ 1 from UNK)
print(len(vocab))

  0%|          | 0/10711 [00:00<?, ?it/s]/Users/idhibhatpankam/Code/courses/NLP-SYS/.venv/lib/python3.12/site-packages/attacut/models/__init__.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this e

4729


In [72]:
vocab_ = defaultdict(lambda: len(vocab_))
for i in tqdm(range(len(data))):
    tokens = attacut.tokenize(data[i][0])
    for token in tokens:
        vocab_[token]
# 5447 unique tokens in entire dataset
print(len(vocab_))

100%|██████████| 13389/13389 [04:25<00:00, 50.42it/s]

5447


In [162]:
oov_count = 0
for i in tqdm(range(len(test_X))):
    tokens = attacut.tokenize(test_X[i])
    for token in tokens:
        if token not in vocab:
            oov_count += 1

# 751 unique tokens in test set but not in training set (OOV)
print(oov_count)

  0%|          | 0/2678 [00:00<?, ?it/s]/Users/idhibhatpankam/Code/courses/NLP-SYS/.venv/lib/python3.12/site-packages/attacut/models/__init__.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this ex

751


In [163]:
def string_to_token_ids(X: np.ndarray):
    new_X = np.zeros(X.shape, dtype=object)
    for i in tqdm(range(len(X))):
        token_ids = []
        tokens = attacut.tokenize(X[i])
        # should we remove any tokens?

        for token in tokens:
            if token in vocab:
                token_ids.append(vocab[token])
            else:
                token_ids.append(vocab["<UNK>"])
        
        new_X[i] = token_ids
    
    return new_X

In [164]:
print(train_X[0])
train_X = string_to_token_ids(train_X)
test_X = string_to_token_ids(test_X)
print(train_X[0])

สอบถามทรูมันนี่มันยังใช่ไม่ได้ครับ


100%|██████████| 2678/2678 [00:55<00:00, 47.85it/s]

[1, 2, 3, 4, 5, 6, 7, 8, 9]


In [187]:
num_docs_with_w = defaultdict(int)
for i in tqdm(range(len(train_X))):
    unique_tokens = set(train_X[i])
    
    for token in unique_tokens:
        num_docs_with_w[token] += 1
# sort token idx
num_docs_with_w = dict(sorted(num_docs_with_w.items(), key=lambda x: x[0]))

idf = np.log(len(train_X) / np.array(list(num_docs_with_w.values())))

100%|██████████| 10711/10711 [00:00<00:00, 142885.92it/s]


In [190]:
len(idf)

4728

In [188]:
num_docs_with_w

{1: 2529,
 2: 120,
 3: 273,
 4: 1233,
 5: 1187,
 6: 174,
 7: 2656,
 8: 2773,
 9: 3020,
 10: 797,
 11: 644,
 12: 826,
 13: 933,
 14: 67,
 15: 316,
 16: 790,
 17: 980,
 18: 1281,
 19: 4,
 20: 145,
 21: 61,
 22: 1380,
 23: 6925,
 24: 421,
 25: 1318,
 26: 2366,
 27: 7,
 28: 1611,
 29: 10,
 30: 368,
 31: 238,
 32: 139,
 33: 438,
 34: 1116,
 35: 323,
 36: 211,
 37: 245,
 38: 3787,
 39: 830,
 40: 836,
 41: 839,
 42: 3507,
 43: 1016,
 44: 1899,
 45: 1576,
 46: 533,
 47: 1568,
 48: 1116,
 49: 3,
 50: 725,
 51: 149,
 52: 1639,
 53: 92,
 54: 1277,
 55: 20,
 56: 955,
 57: 714,
 58: 263,
 59: 432,
 60: 4,
 61: 8,
 62: 827,
 63: 23,
 64: 13,
 65: 116,
 66: 66,
 67: 58,
 68: 657,
 69: 2,
 70: 732,
 71: 723,
 72: 233,
 73: 255,
 74: 1012,
 75: 389,
 76: 26,
 77: 843,
 78: 14,
 79: 344,
 80: 272,
 81: 172,
 82: 10,
 83: 200,
 84: 88,
 85: 1311,
 86: 12,
 87: 79,
 88: 79,
 89: 616,
 90: 377,
 91: 161,
 92: 244,
 93: 8,
 94: 33,
 95: 37,
 96: 309,
 97: 220,
 98: 152,
 99: 130,
 100: 364,
 101: 598,
 102:

In [214]:
idf

array([1.44344728, 4.49153479, 3.66955473, ..., 9.27902653, 9.27902653,
       9.27902653])

In [224]:
def gen_tf_idf_for_sample(sample: list[int]):
    N = len(sample)
    tf = defaultdict(int)
    for token in sample:
        tf[token] += 1
    for token in tf:
        tf[token] = tf[token] / N
    
    tf_idf = np.zeros(len(vocab)) # vocab from only training set
    for token in tf:
        if token < len(vocab)-1:
            tf_idf[token] = tf[token] * idf[token]
    
    return tf_idf

def gen_tf_idf(X: np.ndarray):
    new_X = np.zeros((X.shape[0], len(vocab)))
    for i, sample in enumerate(X):
        new_X[i] = gen_tf_idf_for_sample(sample)
    
    return new_X

In [192]:
print(train_X.shape, test_X.shape, train_y.shape, test_y.shape, len(vocab))

(10711,) (2678,) (10711,) (2678,) 4729


In [225]:
X_train = gen_tf_idf(train_X)
X_test = gen_tf_idf(test_X)
y_train = train_y.copy().astype(int)
y_test = test_y.copy().astype(int)

In [197]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(10711, 4729) (2678, 4729) (10711,) (2678,)


In [245]:
# 100 max epochs took 10s
logmodel = LogisticRegression(class_weight='balanced', max_iter=100, random_state=2025)
logmodel.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', random_state=2025)

In [246]:
start_time = time.time()
predictions = logmodel.predict(X_test)
end_time = time.time()

print(f"Total time: {end_time - start_time}")

Total time: 0.15224909782409668


In [247]:
# 100 epochs acc 64.412%
print("Model Acc. on test data %f%%"
       % ((y_test == predictions).sum() / y_test.shape[0] * 100))

Model Acc. on test data 64.413742%


In [244]:
report = classification_report(y_test, predictions, output_dict=True, digits=2)
report

/Users/idhibhatpankam/Code/courses/NLP-SYS/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/idhibhatpankam/Code/courses/NLP-SYS/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/idhibhatpankam/Code/courses/NLP-SYS/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_

{'0': {'precision': 0.572289156626506,
  'recall': 0.7421875,
  'f1-score': 0.6462585034013606,
  'support': 128.0},
 '1': {'precision': 0.7096774193548387,
  'recall': 0.55,
  'f1-score': 0.6197183098591549,
  'support': 360.0},
 '2': {'precision': 0.7295597484276729,
  'recall': 0.7945205479452054,
  'f1-score': 0.760655737704918,
  'support': 146.0},
 '3': {'precision': 0.6873065015479877,
  'recall': 0.6201117318435754,
  'f1-score': 0.6519823788546255,
  'support': 358.0},
 '4': {'precision': 0.5029239766081871,
  'recall': 0.7413793103448276,
  'f1-score': 0.5993031358885017,
  'support': 116.0},
 '5': {'precision': 0.8379310344827586,
  'recall': 0.5758293838862559,
  'f1-score': 0.6825842696629213,
  'support': 422.0},
 '6': {'precision': 0.359375,
  'recall': 0.46938775510204084,
  'f1-score': 0.40707964601769914,
  'support': 49.0},
 '7': {'precision': 0.8857142857142857,
  'recall': 0.7306397306397306,
  'f1-score': 0.8007380073800738,
  'support': 297.0},
 '8': {'precision'